In [ ]:
import data_simulation_modules as dsm
import plotly.express as px

In [ ]:
temp_df = dsm.generate_demand(nr_days = 28, randomness_level = 3, peak_randomness = 4)
px.scatter(temp_df, x = 'Hour', y = 'Demand', hover_data = ['Specific hour', 'Day of week'])
temp_df.to_csv('updated_demand.csv', index=False)

In [ ]:
temp_df = dsm.generate_demand(nr_days = 50, randomness_level = 3, peak_randomness = 4)
temp_df.to_csv('test_data.csv', index=False)